In [3]:
import torch
from transformers.modeling_bert import BertForMaskedLM
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

tokenizer = BertJapaneseTokenizer.from_pretrained("bert/ja/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask")
model = BertForMaskedLM.from_pretrained("bert/ja/BERT-base_mecab-ipadic-bpe-32k_whole-word-mask")
model.eval()

input_ids = tokenizer.encode(f"""
山田さんが{tokenizer.mask_token}を見たのはこれが初めてでした。巨大だった。
""", return_tensors="pt")

masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]

result = model(input_ids)
result = result[0][:, masked_index].topk(5).indices.tolist()[0]

for r in result:
    output = input_ids[0].tolist()
    output[masked_index] = r
    print(tokenizer.decode(output))

[CLS] 山田さんが 顔 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 面倒 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 目 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 夢 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]
[CLS] 山田さんが 姿 を見たのはこれが初めてでした 。 巨大だった 。 [SEP]


In [5]:
input_ids.shape

torch.Size([1, 23])

In [8]:
text = """
日曜日の16時頃に訪問。
お店の前は通った事がありましたが
今回やっと初訪問。

この日はセールをやっていたようで
いつも外から眺めている印象よりお客さんが多いですね。

様々な種類のお肉が売っておりましたが
既に加工していたり揚げていたりする商品もあったので
そちらを中心に物色します。

コロッケやメンチカツなどもありましたが
唐揚げと串揚げを購入。

唐揚げはグラム数で計り売りで2個で95円でした。
串揚げは150円でした。

その日の夜に特に温めずに頂きましたが
唐揚げは温めなくてもジューシーな味わいで
個人的には非常に好みの味。

串揚げは中に玉葱と挽肉が入っておりましたが
中身はほとんど玉葱のみで、こちらは正直口には合わなかったですね。

それでも唐揚げはさすがお肉屋さんの味といった感想でした。

ご馳走様でした。
"""
tokenizer.encode(text, return_tensors='pt').shape

torch.Size([1, 240])